## Instalar las librerías necesarias y leer un archivo para asignar el nombre de las bandas a proceso de subida a GEE

In [ ]:
%pip install -r requirements.txt

In [2]:
import rasterio

# Ruta al archivo subido
ruta_archivo = 'C:/Users/cespe/Downloads/PSScene/20230102_103243_56_2451_3B_AnalyticMS_SR_8b_harmonized_clip.tif'

# Función para extraer los nombres de las bandas usando rasterio
def obtener_nombres_bandas(ruta_archivo):
    with rasterio.open(ruta_archivo) as dataset:
        nombres_bandas = dataset.descriptions  # Descripciones de las bandas
    return nombres_bandas

# Extraer nombres de las bandas
nombres_bandas = obtener_nombres_bandas(ruta_archivo)
print(nombres_bandas)


('coastal_blue', 'blue', 'green_i', 'green', 'yellow', 'red', 'rededge', 'nir')


## Subir todos los archivos de bucket en GC al ASSET de GEE

In [ ]:
import os
import json
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import google.auth

# Path to the JSON file with the credentials
credentials_path = 'C:/Users/cespe/client_secret_2_775448020170-glgpi5v37ur14of7spbms2gi67g7silt.apps.googleusercontent.com.json'
token_path = 'token.json' 

# Authentication using OAuth 2.0 client credentials
def get_credentials():
    credentials = None
    if os.path.exists(token_path):
        with open(token_path, 'r') as token_file:
            credentials = google.oauth2.credentials.Credentials.from_authorized_user_info(
                json.load(token_file)
            )

    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                credentials_path,
                scopes=['https://www.googleapis.com/auth/cloud-platform']
            )
            credentials = flow.run_local_server(port=0)

        # Guarda las credenciales para la próxima ejecución
        with open(token_path, 'w') as token_file:
            token_file.write(credentials.to_json())

    return credentials

credentials = get_credentials()


print("Authentication successful")

In [2]:
import os
import ee
import rasterio
from datetime import datetime

# Inicializar Earth Engine
ee.Initialize()

In [5]:
from google.oauth2 import service_account
from google.cloud import storage

# Reemplaza con la ruta a tu archivo de clave de cuenta de servicio
service_account_path = 'C:/Users/jesus/ee-jesusc461-02041da0749c.json'

# Configura las credenciales
credentials = service_account.Credentials.from_service_account_file(service_account_path)

# Inicializa el cliente de almacenamiento
storage_client = storage.Client(credentials=credentials)

# Lista los buckets
buckets = list(storage_client.list_buckets())
for bucket in buckets:
    print(bucket.name)


planet_tesis


In [4]:
import ee
ee.Authenticate()
ee.Initialize()


In [1]:
import os
import re
from google.oauth2 import service_account
from google.cloud import storage
import ee
from datetime import datetime

# Configura el archivo de credenciales
service_account_path = 'C:/Users/jesus/ee-jesusc461-02041da0749c.json'
BUCKET_NAME = 'planet_tesis'
BUCKET_PATH = 'GB_1_QGIS/Planet_2'
COLECCION = 'projects/ee-jesusc461/assets/Imagenes_planet_folder/Collection_planet'
NOMBRES_BANDAS = ['coastal_blue', 'blue', 'green_i', 'green', 'yellow', 'red', 'rededge', 'nir']

def initialize_ee():
    try:
        # Inicializa Google Earth Engine
        ee.Initialize()
        print("Google Earth Engine initialized.")
    except Exception as e:
        print(f"Error initializing Earth Engine: {e}")
        raise

def initialize_storage_client():
    creds = service_account.Credentials.from_service_account_file(service_account_path)
    return storage.Client(credentials=creds)

def listar_archivos(bucket_name, prefix):
    storage_client = initialize_storage_client()
    blobs = storage_client.list_blobs(bucket_name, prefix=prefix)
    return [blob.name for blob in blobs if blob.name.endswith('.tif')]

def extract_date_from_filename(filename):
    match = re.search(r'(\d{4})(\d{2})(\d{2})_', filename)
    if match:
        year, month, day = match.groups()
        fecha_formateada = f"{year}-{month}-{day}"
        return fecha_formateada
    else:
        print(f"No date found in filename: {filename}")
        return None

def subir_archivo(uri_gcs, nombre_asset, fecha_formateada):
    asset_id = f"{COLECCION}/{nombre_asset}"
    bands = [{'id': nombre, 'tileset_band_index': i} for i, nombre in enumerate(NOMBRES_BANDAS)]
    start_time = f"{fecha_formateada}T00:00:00Z"  # ISO 8601 format for start time
    end_time = f"{fecha_formateada}T23:59:59Z"    # ISO 8601 format for end time
    
    manifest = {
        'name': asset_id,
        'tilesets': [{
            'sources': [{
                'uris': [uri_gcs]
            }]
        }],
        'bands': bands,
        'startTime': start_time,
        'endTime': end_time
    }
    # print(f"Manifest: {json.dumps(manifest, indent=2)}")
    try:
        task_id = ee.data.newTaskId()[0]
        ee.data.startIngestion(task_id, manifest)
        print(f"Uploaded: {nombre_asset}")
    except Exception as e:
        print(f"Error uploading {nombre_asset}: {e}")

def main():
    initialize_ee()
    archivos = listar_archivos(BUCKET_NAME, BUCKET_PATH)
    for archivo in archivos:
        try:
            archivo_nombre = archivo.split('/')[-1]
            fecha_formateada = extract_date_from_filename(archivo_nombre)
            if fecha_formateada:
                uri_gcs = f"gs://{BUCKET_NAME}/{archivo}"
                nombre_asset = archivo_nombre.split('.')[0]
                subir_archivo(uri_gcs, nombre_asset, fecha_formateada)
        except Exception as e:
            print(f"Error processing file {archivo}: {e}")

if __name__ == "__main__":
    main()


Google Earth Engine initialized.
Uploaded: 20220709_112832_04_2402_3B_AnalyticMS_SR_8b_harmonized_clip
Uploaded: 20220709_112834_33_2402_3B_AnalyticMS_SR_8b_harmonized_clip
Uploaded: 20220715_103926_32_2449_3B_AnalyticMS_SR_8b_harmonized_clip
Uploaded: 20220831_111317_21_2438_3B_AnalyticMS_SR_8b_harmonized_clip
Uploaded: 20220923_111038_92_247a_3B_AnalyticMS_SR_8b_harmonized_clip


In [7]:
import os
import re
from google.oauth2 import service_account
from google.cloud import storage
import ee

# Configura el archivo de credenciales para GCP
service_account_path = 'C:/Users/jesus/ee-jesusc461-02041da0749c.json'

# Configura el bucket y la colección
BUCKET_NAME = 'planet_tesis'
BUCKET_PATH = 'GB_1_QGIS/SAR/Polarimetric_procesed'
COLECCION = 'projects/ee-jesusc461/assets/Imagenes_planet_folder/SAR_collection'
NOMBRES_BANDAS = ['Stokes2_DoLP','Stokes2_LPR','Stokes2_Phi', 'Stokes2_Tau','Stokes2_g0', 'Stokes2_g1', 'Stokes2_g2', 'Stokes2_g3', 'Stokes2_l1', 'Stokes2_l2','alpha', 'entropy', 'entropy_shannon', 'entropy_shannon_norm', 'span']

def initialize_ee():
    try:
        # Inicializa Google Earth Engine
        ee.Initialize()
        print("Google Earth Engine initialized.")
    except Exception as e:
        print(f"Error initializing Earth Engine: {e}")
        raise

def initialize_storage_client():
    creds = service_account.Credentials.from_service_account_file(service_account_path)
    return storage.Client(credentials=creds)

def listar_archivos(bucket_name, prefix):
    storage_client = initialize_storage_client()
    blobs = storage_client.list_blobs(bucket_name, prefix=prefix)
    return [blob.name for blob in blobs if blob.name.endswith('.tif')]

def extract_date_from_filename(filename):
    match = re.search(r'_(\d{8})T', filename)
    if match:
        date_str = match.group(1)
        year, month, day = date_str[:4], date_str[4:6], date_str[6:8]
        fecha_formateada = f"{year}-{month}-{day}"
        return fecha_formateada
    else:
        print(f"No date found in filename: {filename}")
        return None

def subir_archivo(uri_gcs, nombre_asset, fecha_formateada):
    asset_id = f"{COLECCION}/{nombre_asset}"
    bands = [{'id': nombre, 'tileset_band_index': i} for i, nombre in enumerate(NOMBRES_BANDAS)]
    start_time = f"{fecha_formateada}T00:00:00Z"
    end_time = f"{fecha_formateada}T23:59:59Z"
    
    manifest = {
        'name': asset_id,
        'tilesets': [{
            'sources': [{
                'uris': [uri_gcs]
            }]
        }],
        'bands': bands,
        'startTime': start_time,
        'endTime': end_time
    }

    try:
        task_id = ee.data.newTaskId()[0]
        ee.data.startIngestion(task_id, manifest)
        print(f"Uploaded: {nombre_asset}")
    except Exception as e:
        print(f"Error uploading {nombre_asset}: {e}")

def main():
    initialize_ee()
    archivos = listar_archivos(BUCKET_NAME, BUCKET_PATH)
    for archivo in archivos:
        try:
            archivo_nombre = os.path.basename(archivo)
            fecha_formateada = extract_date_from_filename(archivo_nombre)
            if fecha_formateada:
                uri_gcs = f"gs://{BUCKET_NAME}/{archivo}"
                nombre_asset = os.path.splitext(archivo_nombre)[0]
                subir_archivo(uri_gcs, nombre_asset, fecha_formateada)
        except Exception as e:
            print(f"Error processing file {archivo}: {e}")

if __name__ == "__main__":
    main()


Google Earth Engine initialized.
Uploaded: Subset_S1A_IW_SLC__1SDV_20220704T191748_20220704T191815_043955_053F39_0238_Orb_Cal_Deb_Spk_TC
Uploaded: Subset_S1A_IW_SLC__1SDV_20220716T191749_20220716T191816_044130_054475_E70F_Orb_Cal_Deb_Spk_TC
Uploaded: Subset_S1A_IW_SLC__1SDV_20220728T191750_20220728T191817_044305_0549A3_CDA8_Orb_Cal_Deb_Spk_TC
Uploaded: Subset_S1A_IW_SLC__1SDV_20220809T191751_20220809T191817_044480_054ED3_CB5A_Orb_Cal_Deb_Spk_TC
Uploaded: Subset_S1A_IW_SLC__1SDV_20220821T191751_20220821T191818_044655_0554B8_B91E_Orb_Cal_Deb_Spk_TC
Uploaded: Subset_S1A_IW_SLC__1SDV_20220902T191752_20220902T191819_044830_055A9F_1A35_Orb_Cal_Deb_Spk_TC
Uploaded: Subset_S1A_IW_SLC__1SDV_20220914T191752_20220914T191819_045005_056080_DD89_Orb_Cal_Deb_Spk_TC
Uploaded: Subset_S1A_IW_SLC__1SDV_20220926T191752_20220926T191819_045180_056666_83B5_Orb_Cal_Deb_Spk_TC
Uploaded: Subset_S1A_IW_SLC__1SDV_20221008T191753_20221008T191819_045355_056C40_0022_Orb_Cal_Deb_Spk_TC
Uploaded: Subset_S1A_IW_SLC__1S